In [1]:
!pip install selenium

In [2]:
# all imports

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import json
import time

import re 
import pandas as pd

from multiprocessing import Pool

import os

# Testing BeautifulSoup

In [7]:
#URL CENTRAL
#GENERAL URL: 'https://airbnb.com'
NY_GENERAL_SEARCH_URL= 'https://www.airbnb.com/s/New-York--New-York--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=calendar&query=New%20York%2C%20New%20York%2C%20United%20States&place_id=ChIJOwg_06VPwokRYv534QaPC8g&source=structured_search_input_header&search_type=autocomplete_click'

In [14]:
url = 'https://google.com'

In [15]:
answer = requests.get(url)

In [16]:
print(answer.url)
print(answer.status_code)
print(answer.reason)

https://www.google.com/
200
OK


In [17]:
soup = BeautifulSoup(answer.content, 'html.parser')

# START OF AIRBNB CODE 

In [21]:
#Actual Airbnb NY URL 
airbnb_url = 'https://www.airbnb.com/s/New-York--New-York--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=calendar&place_id=ChIJOwg_06VPwokRYv534QaPC8g&query=New%20York%2C%20New%20York%2C%20United%20States&adults=1&search_type=filter_change&room_types%5B%5D=Entire%20home%2Fapt'

In [22]:
b_soup = BeautifulSoup(requests.get(airbnb_url).content, 'html.parser')

In [25]:
listings = b_soup.find_all('div','_gig1e7')

Tags of columns for searching:
Price: 'span', 'a8jt5op dir dir-ltr' 

Title/Name: 'span', 'ts5gl90 tl3qa0j t1nzedvd dir dir-ltr'

Location: 'div', "mj1p6c8 dir dir-ltr"

Ratings: 'div', "sglmc5a dir dir-ltr"

Amentieis: 'div', "mp2hv9t dir dir-ltr"







In [29]:
listings[0].get_text()

'Entire condo in Alphabet CityEast Village with private roof July 2022 (plus)2 guests · 1 bedroom · 1 bed · 1 bathWifi · Kitchen · Air conditioning$75\xa0night$75 per night'

# START OF FUNCTIONS

In [31]:
def scrape_page(page_url):
    """Extracts HTML from a webpage"""
    
    answer = requests.get(page_url)
    content = answer.content
    soup = BeautifulSoup(content, features='html.parser')
    
    return soup

In [32]:
def extract_listing(page_url):
    """Extracts listings from an Airbnb search page"""
    
    page_soup = scrape_page(page_url)
    listings = page_soup.findAll("div", {"class": "_gig1e7"})

    return listings

In [33]:
def extract_element_data(soup, params):
    """Extracts data from a specified HTML element"""
    
    # 1. Find the right tag
    if 'class' in params:
        elements_found = soup.find_all(params['tag'], params['class'])
    else:
        elements_found = soup.find_all(params['tag'])
        
    # 2. Extract text from these tags
    if 'get' in params:
        element_texts = [el.get(params['get']) for el in elements_found]
    else:
        element_texts = [el.get_text() for el in elements_found]
        
    # 3. Select a particular text or concatenate all of them
    tag_order = params.get('order', 0)
    if tag_order == -1:
        output = '**__**'.join(element_texts)
    else:
        output = element_texts[tag_order]
    
    return output

In [34]:
DATA_SEARCH = {
      'Listing_id': {'tag': 'a', 'get': 'target'},
      'Name': { 'tag': 'span', 'class': 'ts5gl90 tl3qa0j t1nzedvd dir dir-ltr'},
      'Price': {'tag': 'span', 'class': 'a8jt5op dir dir-ltr'},
      'Amenities': {'tag':'span', 'class': 'mp2hv9t dir dir-ltr', 'order': -1},
      'Ratings': {'tag': 'div', 'class': "sglmc5a dir dir-ltr", 'order': -1},
      'Location': {'tag': 'div', 'class': "mj1p6c8 dir dir-ltr"},
      'Logistics_URL': {'tag': 'a', 'get': 'href'}
      }

In [36]:
def extract_page_features(soup, rules):
    features_dict = {}
    for feature in rules:
        try:
            features_dict[feature] = extract_element_data(soup, rules[feature])
        except:
            features_dict[feature] = 'empty'
    
    return features_dict

In [37]:
def get_listing_data(airbnb_url, rules) :

    listing_soups = extract_listing(airbnb_url)

    features_list = pd.DataFrame()
    for listing in listing_soups:
        features_dict = {}

        for feature in DATA_SEARCH:
            features_dict[feature] = extract_element_data(listing, rules[feature])
            features_list['Full_URL'] = airbnb_url
        features_list = features_list.append(features_dict, ignore_index=True)

    return features_list

In [38]:
listing_soups = extract_listing('https://www.airbnb.com/s/New-York--New-York--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=may&flexible_trip_dates%5B%5D=june&flexible_trip_dates%5B%5D=july&flexible_trip_dates%5B%5D=august&flexible_trip_dates%5B%5D=september&flexible_trip_dates%5B%5D=october&flexible_trip_dates%5B%5D=november&flexible_trip_dates%5B%5D=december&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&place_id=ChIJOwg_06VPwokRYv534QaPC8g&query=New%20York%2C%20New%20York%2C%20United%20States&adults=1&search_type=unknown')
for element in listing_soups:
    url = element.find('a').get('href')
    full_url= 'https://www.airbnb.com' + str(url)
    print(full_url)

https://www.airbnb.com/rooms/54146235?adults=1&children=0&infants=0&check_in=2022-12-02&check_out=2022-12-04&previous_page_section_name=1000&federated_search_id=d22b9257-f4f4-42a7-9bbf-8666cbd803f7
https://www.airbnb.com/rooms/51687864?adults=1&children=0&infants=0&check_in=2022-10-14&check_out=2022-10-16&previous_page_section_name=1000&federated_search_id=d22b9257-f4f4-42a7-9bbf-8666cbd803f7
https://www.airbnb.com/rooms/43774160?adults=1&children=0&infants=0&check_in=2022-07-15&check_out=2022-07-17&previous_page_section_name=1000&federated_search_id=d22b9257-f4f4-42a7-9bbf-8666cbd803f7
https://www.airbnb.com/rooms/53346269?adults=1&children=0&infants=0&check_in=2022-12-16&check_out=2022-12-18&previous_page_section_name=1000&federated_search_id=d22b9257-f4f4-42a7-9bbf-8666cbd803f7
https://www.airbnb.com/rooms/42563979?adults=1&children=0&infants=0&check_in=2022-10-07&check_out=2022-10-09&previous_page_section_name=1000&federated_search_id=d22b9257-f4f4-42a7-9bbf-8666cbd803f7
https://ww

In [39]:
general_df = get_listing_data('https://www.airbnb.com/s/New-York--New-York--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=may&flexible_trip_dates%5B%5D=june&flexible_trip_dates%5B%5D=july&flexible_trip_dates%5B%5D=august&flexible_trip_dates%5B%5D=september&flexible_trip_dates%5B%5D=october&flexible_trip_dates%5B%5D=november&flexible_trip_dates%5B%5D=december&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&place_id=ChIJOwg_06VPwokRYv534QaPC8g&query=New%20York%2C%20New%20York%2C%20United%20States&adults=1&search_type=unknown', DATA_SEARCH)

In [40]:
general_df

,Full_URL,Amenities,Listing_id,Location,Logistics_URL,Name,Price,Ratings
0,https://www.airbnb.com/s/New-York--New-York--U...,1 guest**__**1 bedroom**__**1 bed**__**1 share...,listing_54146235,Private room in Manhattan,/rooms/54146235?adults=1&children=0&infants=0&...,Micro Hostel in The Heart of Chelsea New York,$51 per night,4.77 (30 reviews)
1,https://www.airbnb.com/s/New-York--New-York--U...,2 guests**__**1 bedroom**__**1 bed**__**1 bath...,listing_51687864,Entire rental unit in Manhattan,/rooms/51687864?adults=1&children=0&infants=0&...,UNTITLED at Freeman - Queen Studio 814,$140 per night,4.86 (28 reviews)
2,https://www.airbnb.com/s/New-York--New-York--U...,2 guests**__**1 bedroom**__**1 bed**__**1 priv...,listing_53843545,Hotel room in Manhattan,/rooms/53843545?adults=1&children=0&infants=0&...,Skyline views in Lower Manhattan,"$180 per night, originally $212",4.48 (21 reviews)
3,https://www.airbnb.com/s/New-York--New-York--U...,1 guest**__**1 bedroom**__**1 bed**__**1 share...,listing_43774160,Private room in Brooklyn,/rooms/43774160?adults=1&children=0&infants=0&...,"Charming, sunny room in artist's apartment",$50 per night,4.80 (111 reviews)
4,https://www.airbnb.com/s/New-York--New-York--U...,1 guest**__**1 bedroom**__**1 bed**__**1 bath*...,listing_53346269,Private room in Upper West Side,/rooms/53346269?adults=1&children=0&infants=0&...,"Clean, quiet, safe room by Central Park",$56 per night,4.80 (30 reviews)
5,https://www.airbnb.com/s/New-York--New-York--U...,3 guests**__**Studio**__**2 beds**__**1 bath**...,listing_42563979,Entire rental unit in Union Square,/rooms/42563979?adults=1&children=0&infants=0&...,Gramercy park Apartment (2beds),$99 per night,4.74 (70 reviews)
6,https://www.airbnb.com/s/New-York--New-York--U...,1 guest**__**1 bedroom**__**1 bed**__**1 share...,listing_14696542,Private room in Brooklyn,/rooms/14696542?adults=1&children=0&infants=0&...,"Bright, Modern, Cozy Room in Brooklyn Artist's...",$67 per night,4.94 (249 reviews)
7,https://www.airbnb.com/s/New-York--New-York--U...,2 guests**__**1 bedroom**__**1 bed**__**1 priv...,listing_47394485,Private room in Bushwick,/rooms/47394485?adults=1&children=0&infants=0&...,Comfortable bedroom with private bathroom,$64 per night,4.84 (96 reviews)
8,https://www.airbnb.com/s/New-York--New-York--U...,1 guest**__**1 bedroom**__**1 bed**__**1 share...,listing_53497691,Private room in Windsor Terrace,/rooms/53497691?adults=1&children=0&infants=0&...,Bright Comfortable Cozy room with skylight for 1,$33 per night,5.0 (17 reviews)
9,https://www.airbnb.com/s/New-York--New-York--U...,4 guests**__**1 bedroom**__**1 bed**__**1 bath...,listing_7609520,Entire rental unit in Hell's Kitchen,/rooms/7609520?adults=1&children=0&infants=0&c...,Hell’s Kitchen Oasis,$189 per night,5.0 (3 reviews)


In [337]:
text_file = open('/Users/Vero/Downloads/Listings_dict')
file_content = text_file.read()
print(file_content)
text_file.close()

{'Name': 'Clean, quiet, safe room by Central Park',
  'Price': '$56 per night',
  'Amenities': '1 guest**__**1 bedroom**__**1 bed**__**1 bath**__**Wifi**__**Kitchen**__**Self check-in**__**Air conditioning',
  'Ratings': '4.80\xa0(30 reviews)',
  'Location': 'Private room in Upper West Side',
  'URL': '/rooms/53346269?adults=1&children=0&infants=0&check_in=2022-12-16&check_out=2022-12-18&previous_page_section_name=1000&federated_search_id=81f522ff-5d71-437e-9726-a1873cd4391f',
  'Listing_id': '53346269'},
 {'Name': 'Gramercy park Apartment (2beds)',
  'Price': '$99 per night',
  'Amenities': '3 guests**__**Studio**__**2 beds**__**1 bath**__**Wifi**__**Kitchen**__**Self check-in**__**Air conditioning',
  'Ratings': '4.74\xa0(70 reviews)',
  'Location': 'Entire rental unit in Union Square',
  'URL': '/rooms/42563979?adults=1&children=0&infants=0&check_in=2022-10-07&check_out=2022-10-09&previous_page_section_name=1000&federated_search_id=81f522ff-5d71-437e-9726-a1873cd4391f',
  'Listing_i

In [43]:
def build_urls(url, listings_per_page=20, pages_per_location=15):
    """Builds links for all search pages for a given location"""
    
    #Since single list, no need for DataFrame
    url_list = []
    for i in range(pages_per_location):
        offset = listings_per_page * i
        url_pagination = url + f'&items_offset={offset}'
        url_list.append(url_pagination)
       
    return url_list

In [44]:
url_list = build_urls('https://www.airbnb.com/s/New-York--New-York--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=calendar&place_id=ChIJOwg_06VPwokRYv534QaPC8g&query=New%20York%2C%20New%20York%2C%20United%20States&adults=1&search_type=filter_change&room_types%5B%5D=Entire%20home%2Fapt')

In [45]:
url_list

['https://www.airbnb.com/s/New-York--New-York--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=calendar&place_id=ChIJOwg_06VPwokRYv534QaPC8g&query=New%20York%2C%20New%20York%2C%20United%20States&adults=1&search_type=filter_change&room_types%5B%5D=Entire%20home%2Fapt&items_offset=0',
 'https://www.airbnb.com/s/New-York--New-York--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=calendar&place_id=ChIJOwg_06VPwokRYv534QaPC8g&query=New%20York%2C%20New%20York%2C%20United%20States&adults=1&search_type=filter_change&room_types%5B%5D=Entire%20home%2Fapt&items_offset=20',
 'https://www.airbnb.com/s/New-York--New-York--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flex

In [47]:
def process_search_pages(url_list, rules):
    features_list = []
    for page in url_list:
        listings = extract_listing(page)
        for listing in listings:
            features = extract_page_features(listing, rules)
            features_list.append(features)

    return features_list

In [159]:
def get_all_listings_data_300(url):
    all_pages_url = build_urls(url)

    all_data = pd.DataFrame()
    for individual_url in all_pages_url:
        all_data = all_data.append(get_listing_data(individual_url, DATA_SEARCH), ignore_index=True)
    return all_data





In [50]:
ny_tribeca = get_all_listings_data_300('https://www.airbnb.com/s/Tribeca/homes?')

In [53]:
ny_tribeca.head(40)

,Full_URL,Amenities,Listing_id,Location,Logistics_URL,Name,Price,Ratings
0,https://www.airbnb.com/s/Tribeca/homes?&items_...,4 guests**__**1 bedroom**__**2 beds**__**1 bat...,listing_49705162,Entire rental unit in SoHo,/rooms/49705162?previous_page_section_name=100...,Modern and charming 1BR in the heart of SoHo,$135 per night,5.0 (4 reviews)
1,https://www.airbnb.com/s/Tribeca/homes?&items_...,2 guests**__**Studio**__**1 bed**__**1 bath**_...,listing_43315528,Entire rental unit in Downtown Manhattan,/rooms/43315528?previous_page_section_name=100...,138 Bowery-Classic Queen Studio,$120 per night,5.0 (4 reviews)
2,https://www.airbnb.com/s/Tribeca/homes?&items_...,1 guest**__**1 bedroom**__**1 bed**__**1 share...,listing_25424961,Private room in SoHo,/rooms/25424961?previous_page_section_name=100...,Charming Glass Room in 1b Apt in heart of SoHo,$65 per night,4.75 (8 reviews)
3,https://www.airbnb.com/s/Tribeca/homes?&items_...,2 guests**__**1 bedroom**__**1 bed**__**1 bath...,listing_36601094,Room in boutique hotel in Downtown Manhattan,/rooms/36601094?previous_page_section_name=100...,"Sohotel, Deluxe Queen",$119 per night,4.27 (15 reviews)
4,https://www.airbnb.com/s/Tribeca/homes?&items_...,2 guests**__**1 bedroom**__**1 bed**__**1 bath...,listing_53398202,Entire rental unit in Downtown Manhattan,/rooms/53398202?previous_page_section_name=100...,Spacious two bedroom apartment in Two Bridges,$101 per night,5.0 (7 reviews)
5,https://www.airbnb.com/s/Tribeca/homes?&items_...,2 guests**__**1 bedroom**__**1 bed**__**1 bath...,listing_14316284,Entire rental unit in West Village,/rooms/14316284?previous_page_section_name=100...,Lovely One Bedroom West Village,$128 per night,4.97 (68 reviews)
6,https://www.airbnb.com/s/Tribeca/homes?&items_...,2 guests**__**Studio**__**1 bed**__**1 bath**_...,listing_52168496,Room in boutique hotel in Downtown Manhattan,/rooms/52168496?previous_page_section_name=100...,Sonder Duane Street | Spacious Queen Room,$285 per night,4.62 (21 reviews)
7,https://www.airbnb.com/s/Tribeca/homes?&items_...,2 guests**__**1 bedroom**__**1 bed**__**1 priv...,listing_39921961,Hotel room in Downtown Manhattan,/rooms/39921961?previous_page_section_name=100...,"The Frederick Hotel, Studio 1 Full Bed",$149 per night,5.0 (4 reviews)
8,https://www.airbnb.com/s/Tribeca/homes?&items_...,2 guests**__**Studio**__**1 bed**__**1 bath**_...,listing_46768272,Entire rental unit in Downtown Manhattan,/rooms/46768272?previous_page_section_name=100...,138 Bowery-Bright Studio,$110 per night,5.0 (4 reviews)
9,https://www.airbnb.com/s/Tribeca/homes?&items_...,4 guests**__**1 bedroom**__**2 beds**__**1 bat...,listing_52169754,Room in boutique hotel in Downtown Manhattan,/rooms/52169754?previous_page_section_name=100...,Sonder Duane Street | King Suite w/ Peloton,$221 per night,


# GOING INTO LISTING DETAILS (SELENIUM/DYNAMIC WEBPAGES

In [54]:
conda install selenium

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\aguiarortega.ve\Anaconda3

  added / updated specs:
    - selenium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.12.0               |   py39haa95532_0        14.5 MB
    selenium-3.141.0           |py39h2bbff1b_1000         815 KB
    ------------------------------------------------------------
                                           Total:        15.2 MB

The following NEW packages will be INSTALLED:

  selenium           pkgs/main/win-64::selenium-3.141.0-py39h2bbff1b_1000

The following packages will be UPDATED:

  conda                               4.10.3-py39haa95532_0 --> 4.12.0-py39haa95532_0




conda-4.12.0         | 14.5 MB   |            |   0% 
conda-4.12.0         | 14.5 MB   |            |   1% 
conda-4.12.0         | 14.5 MB   | #7         |  18% 
conda-4.12.0     

In [100]:
#TRIBECA URL
detail_url='https://www.airbnb.com/rooms/25838923'

In [101]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

ser = Service(r'C:\Users\aguiarortega.ve\AppData\Local\Temp\Temp1_chromedriver_win32.zip\chromedriver.exe')
op = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ser, options=op)

#driver = webdriver.Chrome(r'C:\Users\aguiarortega.ve\AppData\Local\Temp\Temp1_chromedriver_win32.zip\chromedriver.exe')
driver.get(detail_url)

In [74]:
page_detailed = driver.page_source

In [75]:
driver.quit()

In [76]:
detail_soup = BeautifulSoup(page_detailed)

In [77]:
detail_soup.find('span', "ll4r2nl dir dir-ltr")

<span class="ll4r2nl dir dir-ltr">Located in Chinatown, walking distance from Little Italy (10mins), and soho. Nearest subway is on grand street about 9 mins walk. The apartment is on the 4th floor, no elevator. Bedroom has its own private lock. Newly furnished with new mattresses, beddings, and furnitures.<br/><br/>Laundry service is available for $10, wash and dry upon request. Accessible during noon-2pm only.<br/><br/>Please note that there is a key deposit of $30 for two keys to be collected if the guest lost the keys.<br/><br/><span class="_1di55y9">The space</span><br/>This is for one bedroom with queen sized bed. You may use the followings:<br/>-kitchen microwave and fridge. <br/>-Dining table<br/>-shared bathroom</span>

In [114]:
#REVIEWS BUTTON
driver = webdriver.Chrome(service=ser, options=op)
driver.get(detail_url)
time.sleep(5)
element = driver.find_element_by_class_name('_koyt9le')
#element = driver.find_element(By.CLASS_NAME,'_koyt9le')
element.click()
page_detailed = driver.page_source
#driver.close()
detail_soup_clicked = BeautifulSoup(page_detailed)
# driver.find_element_by_xpath("//button[contains(@class='_9n9cos7')]")
#driver.find_element_by_class_name('_le6wlg').click()
#driver.find_element_by_css_selector('[data-testid="homes-pdp-cta-btn"]')

C:\Users\AGUIAR~1.VE\AppData\Local\Temp/ipykernel_10688/3754406871.py:5: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  element = driver.find_element_by_class_name('_koyt9le')


In [116]:
#DESCRIPTION TEXT
detail_soup.find('span', "ll4r2nl dir dir-ltr").get_text()

'Located in Chinatown, walking distance from Little Italy (10mins), and soho. Nearest subway is on grand street about 9 mins walk. The apartment is on the 4th floor, no elevator. Bedroom has its own private lock. Newly furnished with new mattresses, beddings, and furnitures.Laundry service is available for $10, wash and dry upon request. Accessible during noon-2pm only.Please note that there is a key deposit of $30 for two keys to be collected if the guest lost the keys.The spaceThis is for one bedroom with queen sized bed. You may use the followings:-kitchen microwave and fridge. -Dining table-shared bathroom'

In [117]:
button_details = driver.page_source
button_details_clicked = BeautifulSoup(button_details)

In [118]:
detail_soup_clicked.get_text()

"Private Queen bedroom in Chinatown, NYC - Apartments for Rent in New York, New York, United States - AirbnbSkip to contentWe’re sorry, some parts of the Airbnb website don’t work properly without JavaScript enabled.Start your searchPlaces to stayExperiencesLocationNavigate forward to access suggested resultsGo anywhere, anytimeI’m flexibleCheck inAdd datesCheck outAdd datesGuestsAdd guestsSearchBecome a HostPrivate Queen bedroom in Chinatown, NYC4.52 ·188 reviews·New York, United StatesShareSaveShow all photosPhotosAmenitiesReviewsLocation$105\xa0night$105 per night4.52 ·188 reviewsCheck availability Private room in rental unit hosted by\xa0Amiee2 guests · 1 bedroom · 1 bed · 1 shared bathSelf check-inCheck yourself in with the lockbox.Located in Chinatown, walking distance from Little Italy (10mins), and soho. Nearest subway is on grand street about 9 mins walk. The apartment is on the 4th floor, no elevator. Bedroom has its own private lock. Newly furnished with new mattresses, bedd

In [119]:
def extract_review_element(listing_html, params):
    # 1. Find the right tag
    if 'class' in params:
        elements_found = listing_html.find_all(params['tag'], params['class'])
    else:
        elements_found = listing_html.find_all(params['tag'])

    # 2. Extract text from these tags
    if 'get' in params:
        element_texts = [el.get(params['get']) for el in elements_found]
    else:
        element_texts = [el.get_text() for el in elements_found]
    
    # 2. Extract the right element
    tag_order = params.get('order', 0)
    element = elements_found[tag_order]
        
    # 3. Select a particular text or concatenate all of them
    tag_order = params.get('order', 0)
    if tag_order == -1:
        output = '**__**'.join(element_texts)
    else:
        output = element_texts[tag_order]

    return output

In [120]:
extract_review_element(detail_soup_clicked, {'tag': 'span', 'class': 'll4r2nl dir dir-ltr', 'order':-1})

'Located in Chinatown, walking distance from Little Italy (10mins), and soho. Nearest subway is on grand street about 9 mins walk. The apartment is on the 4th floor, no elevator. Bedroom has its own private lock. Newly furnished with new mattresses, beddings, and furnitures.Laundry service is available for $10, wash and dry upon request. Accessible during noon-2pm only.Please note that there is a key deposit of $30 for two keys to be collected if the guest lost the keys.The spaceThis is for one bedroom with queen sized bed. You may use the followings:-kitchen microwave and fridge. -Dining table-shared bathroom**__**Overall, good experience. The communication with the host was quick and succinct.**__**very comfy and clean room in a great location**__**Simple and clean place to stay. Convenient location.**__**Super clean 4 bedroom apartment in the heart of Chinatown. Individual rooms lock and communal bathroom is spotless. Aimee is really responsive and flexible about letting us drop off

In [175]:
def extract_soup_js(listing_url, waiting_time=[5, 1]):
    """Extracts HTML from JS pages: open, wait, click, wait, extract"""
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--blink-settings=imagesEnabled=false')
    driver = webdriver.Chrome(r'C:\Users\aguiarortega.ve\AppData\Local\Temp\Temp1_chromedriver_win32.zip\chromedriver.exe', options=options)
    

    driver.get(listing_url)
    time.sleep(waiting_time[0])
    
    reviews = 0
    try:
        element = driver.find_element_by_class_name('_koyt9le').click()
        reviews = 1
    except:
        pass # REVIEWS BUTTON NOT FOUND 
    
    # if the element is present - click on it
    if reviews == 1:
        for i in range(10): # 10 attempts to scroll to the price button
            try:
                actions = ActionChains(driver)
                driver.execute_script("arguments[0].scrollIntoView(true);", element);
                actions.move_to_element_with_offset(element, 5, 5)
                actions.click().perform()
                break
            except:
                pass
        
    driver.execute_script("window.scrollTo(0, 0);")
    
    time.sleep(waiting_time[1])
    detail_page = driver.page_source

    driver.quit()

    return BeautifulSoup(detail_page, features='html.parser')

In [133]:
def process_detail_page(url):
    soup = extract_soup_js(url, waiting_time=[3, 1])
    rules =  {'tag': 'span', 'class': 'll4r2nl dir dir-ltr', 'order':-1}
    
    features_list = pd.DataFrame()
    features = extract_review_element(soup, rules)
    #Now get Description
    host_description = soup.find('span', 'll4r2nl dir dir-ltr')
    url = soup.find('a').get('href')
    cleaned_ids = ids.group(1).replace('?', '')
    data = {'Reviews': features,'Description': host_description}
    features_list = features_list.append(data, ignore_index=True)
    
    
    

    return features_list

In [130]:
processing = process_detail_page('https://www.airbnb.com/rooms/50616407?source_impression_id=p3_1650221315_%2FpygrzhqpcJEefoF')

C:\Users\AGUIAR~1.VE\AppData\Local\Temp/ipykernel_10688/1687797889.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\aguiarortega.ve\AppData\Local\Temp\Temp1_chromedriver_win32.zip\chromedriver.exe', options=options)
C:\Users\AGUIAR~1.VE\AppData\Local\Temp/ipykernel_10688/1687797889.py:13: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  reviews = driver.find_element_by_class_name('_koyt9le').click()


In [131]:
processing

,Reviews,Description
0,"Located in the heart of Brooklyn's ""Little Hai...","[Located in the heart of Brooklyn's ""Little Ha..."


In [603]:
df = pd.DataFrame(processing, columns=['Description', 'Reviews'])

In [132]:
url_trial = build_urls('https://www.airbnb.com/s/Nolita/homes?')

In [134]:
# try for one page
base_features = process_search_pages(url_trial[4:5], DATA_SEARCH)

In [135]:
base_features

[{'Listing_id': 'listing_54182603',
  'Name': '1 Bedroom Apartment In Lower East Side Manhattan',
  'Price': '$139 per night',
  'Amenities': '5 guests**__**1 bedroom**__**1 bed**__**1 bath**__**Wifi**__**Kitchen**__**Air conditioning',
  'Ratings': '4.65\xa0(17 reviews)',
  'Location': 'Entire rental unit in Chinatown',
  'Logistics_URL': '/rooms/54182603?previous_page_section_name=1000&federated_search_id=7a964817-30b3-4ac2-9a6b-3ac18b0774e1'},
 {'Listing_id': 'listing_37953352',
  'Name': 'Private bedroom in lower Manhattan',
  'Price': '$81 per night',
  'Amenities': '2 guests**__**1 bedroom**__**1 bed**__**1 shared bath**__**Wifi**__**Kitchen**__**Air conditioning',
  'Ratings': '5.0\xa0(4 reviews)',
  'Location': 'Private room in Manhattan',
  'Logistics_URL': '/rooms/37953352?previous_page_section_name=1000&federated_search_id=7a964817-30b3-4ac2-9a6b-3ac18b0774e1'},
 {'Listing_id': 'listing_2339096',
  'Name': 'Cozy bedroom in Lower East Side',
  'Price': '$71 per night',
  'Ame

In [136]:
listings_urls = ['https://www.airbnb.com'+l['Logistics_URL'] for l in base_features]

In [137]:
listings_urls

['https://www.airbnb.com/rooms/54182603?previous_page_section_name=1000&federated_search_id=7a964817-30b3-4ac2-9a6b-3ac18b0774e1',
 'https://www.airbnb.com/rooms/37953352?previous_page_section_name=1000&federated_search_id=7a964817-30b3-4ac2-9a6b-3ac18b0774e1',
 'https://www.airbnb.com/rooms/2339096?previous_page_section_name=1000&federated_search_id=7a964817-30b3-4ac2-9a6b-3ac18b0774e1',
 'https://www.airbnb.com/rooms/37141484?previous_page_section_name=1000&federated_search_id=7a964817-30b3-4ac2-9a6b-3ac18b0774e1',
 'https://www.airbnb.com/rooms/45691383?previous_page_section_name=1000&federated_search_id=7a964817-30b3-4ac2-9a6b-3ac18b0774e1',
 'https://www.airbnb.com/rooms/52007271?previous_page_section_name=1000&federated_search_id=7a964817-30b3-4ac2-9a6b-3ac18b0774e1',
 'https://www.airbnb.com/rooms/47996191?previous_page_section_name=1000&federated_search_id=7a964817-30b3-4ac2-9a6b-3ac18b0774e1',
 'https://www.airbnb.com/rooms/43315428?previous_page_section_name=1000&federated_se

In [140]:
# check the ratio of empty values
def check_empty(features):
    # -2 as we have 2 prices (-1) and 2 specialties (-1)
    cnt, cnt_empty = -2, -2
    for listing in features:
        for key in listing[0]:
            cnt += 1
            if listing[0][key] == 'empty':
                cnt_empty += 1
    return cnt_empty/cnt

In [275]:
def read_all_details(url,start,end):
    all_listing_details = pd.DataFrame()
    url_list = build_urls(url)
    base_features = process_search_pages(url_list[start:end], DATA_SEARCH)
    listings_urls = ['http://www.airbnb.com'+l['Logistics_URL'] for l in base_features]
    for listing in listings_urls:
        all_listing_details['Full_URL'] = listing
        indv_details = process_detail_page(listing)
        all_listing_details = all_listing_details.append(indv_details, ignore_index=True )
        print('still working!')
        
        
        
    return all_listing_details
    
    

In [281]:
#TRIBECA DESCRIPTIONS 
dr_tribeca = read_all_details('https://www.airbnb.com/s/Tribeca/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=calendar&query=New%20York%2C%20New%20York%2C%20United%20States&place_id=ChIJOwg_06VPwokRYv534QaPC8g&source=structured_search_input_header&search_type=autocomplete_click',16,18)

In [154]:
dr_tribeca.to_csv('dr_tribeca.csv')

In [148]:
ny_tribeca.to_csv('ny_tribeca.csv')

In [161]:
#Upper East Side 
ny_ues = get_all_listings_data_300('http://airbnb.com/s/Upper-East-Side-New-York/homes?')

In [179]:
dr_ues = read_all_details('http://airbnb.com/s/Upper-East-Side-New-York/homes?',14)

C:\Users\AGUIAR~1.VE\AppData\Local\Temp/ipykernel_10688/464534709.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\aguiarortega.ve\AppData\Local\Temp\Temp1_chromedriver_win32.zip\chromedriver.exe', options=options)
C:\Users\AGUIAR~1.VE\AppData\Local\Temp/ipykernel_10688/464534709.py:14: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  element = driver.find_element_by_class_name('_koyt9le').click()


0
still working!
1
still working!
2
still working!
3
still working!
4
still working!
5
still working!
6
still working!
7
still working!
8
still working!
9
still working!
10
still working!
11
still working!
12
still working!
13
still working!
14
still working!
15
still working!
16
still working!
17
still working!
18
still working!
19
still working!


In [180]:
dr_ues

,Full_URL,Reviews,Description
0,http://www.airbnb.com/rooms/50199150?previous_...,Show up and start living from day one in Uptow...,[Show up and start living from day one in Upto...
1,http://www.airbnb.com/rooms/50199150?previous_...,"Well located, central to everything. Roosevelt...","[Well located, central to everything. Roosevel..."
2,http://www.airbnb.com/rooms/50199150?previous_...,"This furnished apartment is a stunning, newly ...","[This furnished apartment is a stunning, newly..."
3,http://www.airbnb.com/rooms/50199150?previous_...,"Gentrification hasn’t erased Harlem’s charm, a...","[Gentrification hasn’t erased Harlem’s charm, ..."
4,http://www.airbnb.com/rooms/50199150?previous_...,Impressive executive apartment right in the he...,[Impressive executive apartment right in the h...
5,http://www.airbnb.com/rooms/50199150?previous_...,"A comfortable, safe, and sizable shared apartm...","[A comfortable, safe, and sizable shared apart..."
6,http://www.airbnb.com/rooms/50199150?previous_...,This large one bedroom apartment was recently ...,[This large one bedroom apartment was recently...
7,http://www.airbnb.com/rooms/50199150?previous_...,"Welcome to NYC, the city that never sleeps! My...","[Welcome to NYC, the city that never sleeps! M..."
8,http://www.airbnb.com/rooms/50199150?previous_...,"Stay safe, stay Jurny! No human interaction is...","[Stay safe, stay Jurny! No human interaction i..."
9,http://www.airbnb.com/rooms/50199150?previous_...,Apartment is located on UES of Manhattan! Only...,"[Apartment is located on UES of Manhattan! , [..."


In [182]:
dr_ues.to_csv('dr_ues.csv')

In [194]:
ny_gramercy = get_all_listings_data_300('http://airbnb.com/s/Gramercy-Park-New-York/homes?')

In [195]:
ny_gramercy

,Full_URL,Amenities,Listing_id,Location,Logistics_URL,Name,Price,Ratings
0,http://airbnb.com/s/Gramercy-Park-New-York/hom...,1 guest**__**1 bedroom**__**1 bed**__**4 share...,listing_42682643,Room in boutique hotel in Manhattan,/rooms/42682643?previous_page_section_name=100...,SSB 12. SINGLE SHARED BATHROOM,$59 per night,
1,http://airbnb.com/s/Gramercy-Park-New-York/hom...,3 guests**__**Studio**__**2 beds**__**1 bath**...,listing_42563979,Entire rental unit in Union Square,/rooms/42563979?previous_page_section_name=100...,Gramercy park Apartment (2beds),$85 per night,4.73 (71 reviews)
2,http://airbnb.com/s/Gramercy-Park-New-York/hom...,2 guests**__**1 bedroom**__**1 bed**__**1 shar...,listing_42672172,Private room in Union Square,/rooms/42672172?previous_page_section_name=100...,Near Gramercy Park/ Private Entrance,$75 per night,4.54 (89 reviews)
3,http://airbnb.com/s/Gramercy-Park-New-York/hom...,3 guests**__**Studio**__**2 beds**__**1 bath**...,listing_42746488,Entire rental unit in Union Square,/rooms/42746488?previous_page_section_name=100...,Best Quiet Apartment / Union Square,$92 per night,4.67 (108 reviews)
4,http://airbnb.com/s/Gramercy-Park-New-York/hom...,1 guest**__**1 bedroom**__**1 bed**__**1 share...,listing_3757461,Private room in Manhattan,/rooms/3757461?previous_page_section_name=1000...,Manhattan Sun filled-near NYU & Bellevue Hospi...,$73 per night,4.70 (222 reviews)
...,...,...,...,...,...,...,...,...
295,http://airbnb.com/s/Gramercy-Park-New-York/hom...,2 guests**__**1 bedroom**__**1 bed**__**1 priv...,listing_605721169816342867,Private room in Manhattan,/rooms/605721169816342867?previous_page_sectio...,Lovely Flatiron Smart Studio,$117 per night,
296,http://airbnb.com/s/Gramercy-Park-New-York/hom...,2 guests**__**1 bedroom**__**1 bed**__**1 priv...,listing_560058618593938132,Private room in Manhattan,/rooms/560058618593938132?previous_page_sectio...,Beautiful Private Studio | Flatiron,$114 per night,
297,http://airbnb.com/s/Gramercy-Park-New-York/hom...,3 guests**__**1 bedroom**__**1 bed**__**1 bath...,listing_609192394486222637,Entire serviced apartment in Manhattan,/rooms/609192394486222637?previous_page_sectio...,All-inclusive apartment home | 1 BR in New York,$338 per night,
298,http://airbnb.com/s/Gramercy-Park-New-York/hom...,2 guests**__**1 bedroom**__**1 bed**__**1 shar...,listing_598573051901235365,Private room in Kips Bay,/rooms/598573051901235365?previous_page_sectio...,Spacious -bedroom rental in New York,$100 per night,


In [197]:
dr_gramercy = read_all_details('http://airbnb.com/s/Gramercy-Park-New-York/homes?',12)

['http://www.airbnb.com/rooms/13864551?previous_page_section_name=1000&federated_search_id=c405ecc2-e077-4643-a006-3ffa9f271bf6', 'http://www.airbnb.com/rooms/6014926?previous_page_section_name=1000&federated_search_id=c405ecc2-e077-4643-a006-3ffa9f271bf6', 'http://www.airbnb.com/rooms/49512986?previous_page_section_name=1000&federated_search_id=c405ecc2-e077-4643-a006-3ffa9f271bf6', 'http://www.airbnb.com/rooms/44320456?previous_page_section_name=1000&federated_search_id=c405ecc2-e077-4643-a006-3ffa9f271bf6', 'http://www.airbnb.com/rooms/17602108?previous_page_section_name=1000&federated_search_id=c405ecc2-e077-4643-a006-3ffa9f271bf6', 'http://www.airbnb.com/rooms/177606?previous_page_section_name=1000&federated_search_id=c405ecc2-e077-4643-a006-3ffa9f271bf6', 'http://www.airbnb.com/rooms/4377929?previous_page_section_name=1000&federated_search_id=c405ecc2-e077-4643-a006-3ffa9f271bf6', 'http://www.airbnb.com/rooms/753622?previous_page_section_name=1000&federated_search_id=c405ecc2-e07

C:\Users\AGUIAR~1.VE\AppData\Local\Temp/ipykernel_10688/464534709.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\aguiarortega.ve\AppData\Local\Temp\Temp1_chromedriver_win32.zip\chromedriver.exe', options=options)
C:\Users\AGUIAR~1.VE\AppData\Local\Temp/ipykernel_10688/464534709.py:14: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  element = driver.find_element_by_class_name('_koyt9le').click()


0
still working!
1
still working!
2
still working!
3
still working!
4
still working!
5
still working!
6
still working!
7
still working!
8
still working!
9
still working!
10
still working!
11
still working!
12
still working!
13
still working!
14
still working!
15
still working!
16
still working!
17
still working!
18
still working!
19
still working!
20
still working!
21
still working!
22
still working!
23
still working!
24
still working!
25
still working!
26
still working!
27
still working!
28
still working!
29
still working!
30
still working!
31
still working!
32
still working!
33
still working!
34
still working!
35
still working!
36
still working!
37
still working!
38
still working!
39
still working!
40
still working!
41
still working!
42
still working!
43
still working!
44
still working!
45
still working!
46
still working!
47
still working!
48
still working!
49
still working!
50
still working!
51
still working!
52
still working!
53
still working!
54
still working!
55
still working!
56

In [198]:
dr_gramercy

,Full_URL,Reviews,Description
0,http://www.airbnb.com/rooms/35379061?previous_...,The apartment is among the best and most uniqu...,[The apartment is among the best and most uniq...
1,http://www.airbnb.com/rooms/35379061?previous_...,Cozy 3rd fl.studio walk-up in cosmopolitan e. ...,[Cozy 3rd fl.studio walk-up in cosmopolitan e....
2,http://www.airbnb.com/rooms/35379061?previous_...,Hi! My apartment is located in the heart of th...,[Hi! My apartment is located in the heart of t...
3,http://www.airbnb.com/rooms/35379061?previous_...,Our apartment is located in the center of Manh...,[Our apartment is located in the center of Man...
4,http://www.airbnb.com/rooms/35379061?previous_...,Welcome home! East 7th Street Studio in center...,[Welcome home! East 7th Street Studio in cente...
5,http://www.airbnb.com/rooms/35379061?previous_...,"The spaceMODERN,SPACIOUS ALCOVE STUDIO THAT CA...","[[The space], [], MODERN,SPACIOUS ALCOVE STUDI..."
6,http://www.airbnb.com/rooms/35379061?previous_...,Located in the heart of the East Village on a ...,[Located in the heart of the East Village on a...
7,http://www.airbnb.com/rooms/35379061?previous_...,Covid Update: Please provide proof (at least w...,[Covid Update: Please provide proof (at least ...
8,http://www.airbnb.com/rooms/35379061?previous_...,Your room charge will be collected by Airbnb w...,[Your room charge will be collected by Airbnb ...
9,http://www.airbnb.com/rooms/35379061?previous_...,Our bright and airy duplex is located in the h...,[Our bright and airy duplex is located in the ...


In [637]:
#NY_100 (0:4)
ny_100 = read_all_details(NY_GENERAL_SEARCH_URL)

In [638]:
ny_100

,Full_URL,Description,Reviews
0,http://www.airbnb.com/rooms/17093180?previous_...,[Passing through New York City for a short vis...,Passing through New York City for a short visi...
1,http://www.airbnb.com/rooms/17093180?previous_...,[the room it is in a 2 Beds Apartment to be sh...,the room it is in a 2 Beds Apartment to be sha...
2,http://www.airbnb.com/rooms/17093180?previous_...,[Quiet room available in an artist's apartment...,Quiet room available in an artist's apartment....
3,http://www.airbnb.com/rooms/17093180?previous_...,"[The biggest room in Brooklyn, furnished with ...","The biggest room in Brooklyn, furnished with Q..."
4,http://www.airbnb.com/rooms/17093180?previous_...,"[Welcome to Hotel 17 Extended Stay Hotel., [],...",Welcome to Hotel 17 Extended Stay Hotel.With i...
...,...,...,...
75,http://www.airbnb.com/rooms/17093180?previous_...,[Clean and beautifully furnished studio apartm...,Clean and beautifully furnished studio apartme...
76,http://www.airbnb.com/rooms/17093180?previous_...,"[I am easy going, accessible and health profes...","I am easy going, accessible and health profess..."
77,http://www.airbnb.com/rooms/17093180?previous_...,[This space is shared with one person but you...,This space is shared with one person but you ...
78,http://www.airbnb.com/rooms/17093180?previous_...,[A charming one bedroom apartment in the heart...,A charming one bedroom apartment in the heart ...


In [644]:
ny_100_2 = read_all_details(NY_GENERAL_SEARCH_URL,5,9)

In [647]:
ny_80 = read_all_details(NY_GENERAL_SEARCH_URL,10,14)

In [648]:
ny_80.to_csv('ny_80.csv')

In [639]:
ny_100.to_csv('ny_100.csv')

In [646]:
ny_100_2.to_csv('ny_100_2.csv')

In [640]:
ny_gen_300.to_csv('ny_gen_300.csv')